In [1]:
import pandas as pd
df = pd.read_csv("../../all_data/crypto/coinbase_usd_all_samples_mean_per_hour.csv")
df.tail()

,Unnamed: 0,hours,prices_usd,volumes_usd,prices_delta,prices_delta_sign,volumes_delta,volumes_delta_sign
30252,30252,30253,8737.940930,3.148853e+06,-20.639571,-1,5.854928e+05,1
30253,30253,30254,8750.566713,3.778666e+06,12.625784,1,6.298132e+05,1
30254,30254,30255,8748.972231,2.641025e+06,-1.594482,-1,-1.137641e+06,-1
30255,30255,30256,8834.477107,1.201254e+07,85.504876,1,9.371513e+06,1
30256,30256,30257,8715.337531,6.909141e+06,-119.139576,-1,-5.103397e+06,-1


In [2]:
# on calcule le ratio du delta
df["prices_delta_ratio"] = df.prices_delta / df.prices_usd
df["volumes_delta_ratio"] = df.volumes_delta / df.volumes_usd
df.head()

,Unnamed: 0,hours,prices_usd,volumes_usd,prices_delta,prices_delta_sign,volumes_delta,volumes_delta_sign,prices_delta_ratio,volumes_delta_ratio
0,0,1,370.0,9.82555,46.666667,1,0.12555,1,0.126126,0.012778
1,1,2,370.0,0.00000,0.000000,0,-9.82555,-1,0.000000,-inf
2,2,3,370.0,0.00000,0.000000,0,0.00000,0,0.000000,NaN
3,3,4,370.0,0.00000,0.000000,0,0.00000,0,0.000000,NaN
4,4,5,370.0,0.00000,0.000000,0,0.00000,0,0.000000,NaN


In [3]:
X = []
Y = []
w_size = 720

def vectorize(hour, df):
    start_hour = hour - w_size
    end_hour = hour
    vec = df.prices_delta_ratio.values[start_hour:end_hour]
    return vec

# for hour in df.hours[1000:1500:1]:
for hour in df.hours[w_size:-2:1]:
    hour_output = hour + 1
    vec = vectorize(hour, df)
    y = df.prices_delta_sign[hour_output]
    X.append(vec)
    Y.append(y)
    if hour%10000 == 0:
        print(hour, y)

10000 -1
20000 1
30000 1


In [4]:
data_df = pd.DataFrame(X)
data_df["y"] = Y

In [7]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np

# on split les données en apprentissage/test
X_train, X_test, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    shuffle=True, 
    test_size=0.2, 
    train_size=None)

In [8]:
clf = GaussianNB()
print("start training")
clf.fit(X_train, Y_train)
print("done")
predicted = clf.predict(X_test)
print("correct ones", np.sum(predicted == Y_test))
print("false ones", np.sum(predicted != Y_test))
print("ratio:", np.mean(predicted == Y_test))

start training
done
correct ones 2763
false ones 3144
ratio: 0.46775012696800405
